In [1]:
!pip install transformers

     |████████████████████████████████| 2.5MB 8.1MB/s 
     |████████████████████████████████| 3.3MB 42.2MB/s 
     |████████████████████████████████| 901kB 49.4MB/s 


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu
import nltk
import numpy as np
import pandas as pd

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id = tokenizer.eos_token_id)

In [9]:
df = pd.read_excel('Data_SOC.xlsx')
df = pd.DataFrame(df)
df1 = pd.DataFrame(columns={'Finance','Sc_Literature','Music/Entertainment'})
df1.head()

,Music/Entertainment,Finance,Sc_Literature


In [11]:
i = 0
while( i < len(df)):
  content = df['Sc_Literature'][i]
  input_ids = tokenizer.encode(content[:100], return_tensors='pt')
  output = model.generate(input_ids, max_length = 1000, do_sample = True, top_k = 100, top_p = 0.9)

  reference = tokenizer.decode(output[0], skip_special_tokens= True).split()

  len1 = len(content)
  len2 = len(reference)
  if(len1 >= len2):
    hypothesis = content.split()[:len2]
  else:
    reference = reference[:len1]
    hypothesis = content.split()

  smoothie = SmoothingFunction().method7
  BLEUscore = nltk.translate.bleu_score.corpus_bleu([reference], [hypothesis], smoothing_function=smoothie)
  df1.loc[i,'Sc_Literature'] = BLEUscore
  print(BLEUscore)
  i += 1

0.36543538581879464
0.3654130719887559
0.365090277220685
0.3636005413293584
0.3640519332520572
0.3649279282836811
0.3636005413293584
0.3655038242770524
0.3652519273037479
0.36592350714044847


In [12]:
i = 0
while( i != len(df)):
  content = df['Finance'][i]
  input_ids = tokenizer.encode(content[:100], return_tensors='pt')
  output = model.generate(input_ids, max_length = 1000, do_sample = True, top_k = 100, top_p = 0.9)

  reference = tokenizer.decode(output[0], skip_special_tokens= True).split()
  
  len1 = len(content)
  len2 = len(reference)
  if(len1 >= len2):
    hypothesis = content.split()[:len2]
  else:
    reference = reference[:len1]
    hypothesis = content.split()

  smoothie = SmoothingFunction().method7
  BLEUscore = nltk.translate.bleu_score.corpus_bleu([reference], [hypothesis], smoothing_function=smoothie)
  df1.loc[i,'Finance'] = BLEUscore
  print(BLEUscore)
  i += 1

0.3642517773942746
0.3663022407160805
0
0
0.3657922229596879
0.36601028429925375
0.36557036943582855
0.36592350714044847
0.3491969342126689
0.366874131073038


In [13]:
i = 0
while( i != len(df)):
  content = df['Music/Entertainment'][i]
  input_ids = tokenizer.encode(content[:100], return_tensors='pt')
  output = model.generate(input_ids, max_length = 1000, do_sample = True, top_k = 100, top_p = 0.9)

  reference = tokenizer.decode(output[0], skip_special_tokens= True).split()

  len1 = len(content)
  len2 = len(reference)
  if(len1 >= len2):
    hypothesis = content.split()[:len2]
  else:
    reference = reference[:len1]
    hypothesis = content.split()

  smoothie = SmoothingFunction().method7
  BLEUscore = nltk.translate.bleu_score.corpus_bleu([reference], [hypothesis], smoothing_function=smoothie)
  df1.loc[i,'Music/Entertainment'] = BLEUscore
  print(BLEUscore)
  i += 1

0.36459426919010207
0.36826655206616193
0.3582030837768331
0.3657922229596879
0.36574403671838307
0.3688163311050085
0.36716660741460677
0.36763133738393583
0.35706319452284657
0.3661755750852203


In [15]:
df1.to_csv(r'BLEU_Scores_GPT-2.csv', index=False)